In [1]:
import pandas as pd
import tarfile
import os
import shutil

Unzip tar file containing the data

In [2]:
fname = "DLCV_logo_project.tar.gz"
destination_dir = "./"

tar = tarfile.open(fname)
tar.extractall(destination_dir)
tar.close()

Import annotations

In [ ]:
seed = 12
df = pd.read_csv("./DLCV_logo_project/annot_train.csv")
df = df.sample(frac=1, random_state=seed)

,photo_filename,width,height,class,xmin,ymin,xmax,ymax
30778,la_956599681810104253_20150405.jpg,640,640,Ralph Lauren Corporation,108,2,510,405
33150,la_2017_1640035640777963014_20171103.jpg,1080,1350,Apple Inc.,525,1165,590,1256
1793,chicago_1469084333612408394_20170312.jpg,1080,1080,Nike,665,627,814,726
29836,cleveland_1548212134353023654_20170630.jpg,750,750,Ralph Lauren Corporation,68,1,696,266
28609,sf_1284305827758489800_20160630.jpg,1080,1349,Starbucks,397,607,724,984
...,...,...,...,...,...,...,...,...
28711,sf_1383666054890738165_20161114.jpg,1080,1080,Starbucks,429,536,655,761
6024,la_2017_1582765993539933240_20170816.jpg,320,400,Nike,68,153,151,211
23608,chicago_1253867671820073621_20160519.jpg,1080,1080,The North Face,721,976,816,1048
37650,houston_1445043296636508963_20170207.jpg,1080,1080,NFL,479,141,546,249


Extract annotations of the 5 compulsory logos

In [ ]:
comp_df = df.copy()
comp_df.head()

,photo_filename,width,height,class,xmin,ymin,xmax,ymax
0,charlotte_1019088214340003257_20150630.jpg,640,640,Nike,339,394,381,439
1,charlotte_1056018576661922536_20150820.jpg,640,640,Nike,497,0,574,30
2,charlotte_1057610050380125945_20150823.jpg,1080,1080,Nike,353,406,473,477
3,charlotte_1061715143228072770_20150828.jpg,1080,1349,Nike,13,51,330,198
4,charlotte_1081463278280976819_20150925.jpg,612,612,Nike,485,30,580,69


Remove space and special characters from logo names

In [17]:
comp_df["class"] = comp_df["class"].apply(lambda x: ''.join(char for char in x if char.isalnum()))
comp_df

<ipython-input-17-d944404703a6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comp_df["class"] = comp_df["class"].apply(lambda x: ''.join(char for char in x if char.isalnum()))


,photo_filename,width,height,class,xmin,ymin,xmax,ymax
0,charlotte_1019088214340003257_20150630.jpg,640,640,Nike,339,394,381,439
1,charlotte_1056018576661922536_20150820.jpg,640,640,Nike,497,0,574,30
2,charlotte_1057610050380125945_20150823.jpg,1080,1080,Nike,353,406,473,477
3,charlotte_1061715143228072770_20150828.jpg,1080,1349,Nike,13,51,330,198
4,charlotte_1081463278280976819_20150925.jpg,612,612,Nike,485,30,580,69
...,...,...,...,...,...,...,...,...
24600,sf_1844169196180010544_20180812.jpg,1080,1080,TheNorthFace,691,975,754,1033
24601,sf_1844972039627225248_20180813.jpg,1080,1350,TheNorthFace,21,901,131,972
24602,sf_1863555196800211015_20180908.jpg,1080,1137,TheNorthFace,712,871,828,987
24603,sf_1864449808673458749_20180909.jpg,1080,1080,TheNorthFace,602,216,713,288


Create folder storing data 

In [18]:
path = "./data"

try:
    os.mkdir(path)
except:
    pass

Create a folder containing the training images of compulsory logos only

In [19]:
dest_folder = "./data/train"
source_folder = "./DLCV_logo_project/train/"

try:
    os.mkdir(dest_folder)
except:
    pass

filenames = comp_df["photo_filename"].values

for f in filenames:
    try:
        shutil.copy(source_folder + f, dest_folder)
    except:
        pass

In [20]:
n_files = len([f for f in os.listdir(dest_folder)])
print(f"Train folder contains {n_files} images.")

Train folder contains 20974 images.


Split training images into 5 subfolders to perform annotations on Roboflow

In [21]:
# Create dictionary containing subfolder names as keys and a list of files as value
subfolders = ["Reb", "Elio", "Eline", "AleL", "AleC"]
subfolders_dict = {}
subfolder_size = int(len(comp_df) * 0.2)
temp_df = comp_df.copy()

for s in subfolders[:-1]:
    subfolder_pics = temp_df.sample(n=subfolder_size, random_state=seed)["photo_filename"].values
    subfolders_dict[s] = subfolder_pics
    temp_df = temp_df[~(temp_df["photo_filename"].isin(subfolder_pics))]

subfolders_dict[subfolders[-1]] = temp_df["photo_filename"].values

Nike            0.455395
Adidas          0.373908
UnderArmour     0.069498
Puma            0.051006
TheNorthFace    0.050193
Name: class, dtype: float64
Nike            0.456615
Adidas          0.378988
UnderArmour     0.070108
TheNorthFace    0.051006
Puma            0.043284
Name: class, dtype: float64
Nike            0.434871
Adidas          0.391181
UnderArmour     0.076204
TheNorthFace    0.052632
Puma            0.045113
Name: class, dtype: float64
Nike            0.442999
Adidas          0.393213
UnderArmour     0.068076
TheNorthFace    0.049990
Puma            0.045722
Name: class, dtype: float64


In [11]:
# Create subfolders directory
try:
    os.mkdir("./data/subfolders")
except:
    pass

In [12]:
# For each key in the subfolders dictionary
source_folder = "./data/train/"
for folder_name, filenames in subfolders_dict.items():
    # 1. Create subfolder corresponding to the key
    dest_folder = "./data/subfolders/" + folder_name
    try:
        os.mkdir(dest_folder)
    except:
        pass
    # 2. Copy files contained in the dictionary value corresponding to that key to the subfolder
    for f in filenames:
        try:
            shutil.copy(source_folder + f, dest_folder)
        except:
            pass


Zip the subfolders directory

In [68]:
shutil.make_archive("./subfolders", "zip", "./data/subfolders")

'c:\\Users\\dioge\\Desktop\\Uni\\2nd Year\\1st Sem\\Computer Vision\\Logo-Detection\\Autocomposer\\subfolders.zip'